# Ensembles - RE

In [ ]:
import os, sys
path = os.getcwd()+"/../.."
sys.path.append(path)    #give here the path to the folder containing the package,default does not work!!
print(os.getcwd())

import numpy as np
import matplotlib.pyplot as plt

params ={"figure.facecolor": "w",
         "font.size": 14}
plt.rcParams.update(params)

%matplotlib inline

#Ensembler
from ensembler.samplers.newtonian import positionVerletIntegrator
from ensembler.potentials.OneD import harmonicOscillatorPotential, envelopedPotential
#from ensembler.integrator.newtonian import positionVerletIntegrator
from ensembler.system.basic_system import system as basic_system

# Simulated Tempering

Sugita, Okinamura?

## Basic Definitions

In [ ]:
#Potential:
potential = harmonicOscillatorPotential()
#Integrator
integrator =positionVerletIntegrator()# langevinIntegrator()

system = basic_system(sampler=integrator, potential=potential, start_position=5)

#Ensemble
from ensembler.ensemble import replica_exchange

trials = 10
steps_between_trials = 50
temperature_range = np.linspace(298, 550, 4)

simulated_tempering = replica_exchange.temperatureReplicaExchange(system=system, temperature_Range=temperature_range, steps_between_trials=steps_between_trials)
simulated_tempering.replicas

In [ ]:
simulated_tempering.get_replicas_current_states()

In [ ]:
reps = simulated_tempering.replicas

for repID, replica in reps.items():
    print(repID)
    replica.simulate(100, withdrawTraj=True)
print(vars(replica))

In [ ]:
trajs = simulated_tempering.get_trajectories()

positions = np.linspace(-10,10)
fig, axes = plt.subplots(ncols=len(trajs), figsize=[16,9])

for traj, ax in zip(trajs, axes):
    ax.plot(positions, simulated_tempering.replicas[traj].potential.ene(positions), c="grey")
    ax.scatter(trajs[traj].position, trajs[traj].totPotEnergy)
    min_e = min(trajs[traj].totEnergy)
    #ax.set_ylim([min_e,min_e+10])
    ax.set_title("replica"+str(traj))

trajs[3].head()
trajs = simulated_tempering.get_trajectories()

positions = np.linspace(-10,10)
fig, axes = plt.subplots(ncols=len(trajs), figsize=[16,9])

for traj, ax in zip(trajs, axes):
    ax.plot(positions, simulated_tempering.replicas[traj].potential.ene(positions), c="grey")
    ax.scatter(trajs[traj].position, trajs[traj].totPotEnergy)
    min_e = min(trajs[traj].totEnergy)
    #ax.set_ylim([min_e,min_e+10])
    ax.set_title("replica"+str(traj))

trajs[2].head()


In [ ]:
simulated_tempering.simulate(trials, reset_ensemble=True)
#ensemble.exchange_information

In [ ]:
trajs = simulated_tempering.get_trajectories()

positions = np.linspace(-10,10)
fig, axes = plt.subplots(ncols=len(trajs), figsize=[16,9])

for traj, ax in zip(trajs, axes):
    ax.plot(positions, simulated_tempering.replicas[traj].potential.ene(positions), c="grey")
    ax.scatter(trajs[traj].position, trajs[traj].totPotEnergy)
    min_e = min(trajs[traj].totEnergy)
    #ax.set_ylim([min_e,min_e+10])
    ax.set_title("replica"+str(traj))

trajs[3].head()


In [ ]:
trajs[3].head()


In [ ]:
simulated_tempering.replicas[0]

In [ ]:
simulated_tempering.exchange_information

# Visualisation

In [ ]:
trajs = simulated_tempering.get_trajectories()

positions = np.linspace(-10,10)
fig, axes = plt.subplots(ncols=len(trajs), figsize=[16,9])

for traj, ax in zip(trajs, axes):
    ax.plot(positions, simulated_tempering.replicas[traj].potential.ene(positions), c="grey")
    ax.scatter(trajs[traj].position, trajs[traj].totPotEnergy)
    min_e = min(trajs[traj].totEnergy)
    #ax.set_ylim([min_e,min_e+5])
    ax.set_title("replica"+str(traj))


In [ ]:
trajs = simulated_tempering.get_trajectories()

positions = np.linspace(-10,10)
fig, axes = plt.subplots(ncols=len(trajs), figsize=[16,9], sharey=True)

for traj, ax in zip(trajs, axes):
    ax.boxplot(trajs[traj].totKinEnergy)
    min_e = min(trajs[traj].totEnergy)
    ax.set_title("replica"+str(traj))


In [ ]:

fig, ax = plt.subplots(ncols=1, figsize=[16,9])
enes = [trajs[x].totPotEnergy	for x in sorted(trajs)] 
min_e = np.min(enes)
max_e = np.max(enes)
trials = 10
for ind, ene in enumerate(enes):
    ax.plot(ene, label="replica"+str(ind))
ax.vlines([steps_between_trials*x for x in range(trials)], ymin=min_e, ymax=max_e, label="exchange", lw=0.1)
ax.set_title("replica potE timeseries")
ax.set_ylim(1.1*min_e, 1.1*max_e)
ax.legend()

In [ ]:
stats= simulated_tempering.exchange_information
replicas = np.unique(simulated_tempering.exchange_information.replicaID)
trials = np.unique(simulated_tempering.exchange_information.nExchange)

import itertools as it

fig, ax = plt.subplots(ncols=1, figsize=[16,9])

replica_positions = {}
for replica in replicas:
    replica_positions.update({replica: stats.loc[stats.replicaID==replica].replicaPositionI})

    x = trials
    y = replica_positions[replica]

    ax.plot(x,y , label="replica_"+str(replica))

#plt.yticks(replicas+1, reversed(replicas+1))
ax.set_yticks(ticks=replicas)
ax.set_yticklabels(labels=replicas)

ax.set_ylabel("replica Positions")
ax.set_xlabel("trials")
ax.set_title("Replica exchange transitions")
if(len(replicas)<10): plt.legend()

# Hamiltonian Replica Exchange

## RE-EDS

Replica Exchange - Enveloping Distribution Sampling (RE-EDS) is a combined Method, that efficiently can calculate free energy differences between mutliple end-states

In [ ]:
import os, sys
path = os.getcwd()+"/../.."
sys.path.append(path)    #give here the path to the folder containing the package,default does not work!!
print(os.getcwd())

import numpy as np
import matplotlib.pyplot as plt

params ={"figure.facecolor": "w",
         "font.size": 14}
plt.rcParams.update(params)

%matplotlib inline

#Ensembler
from ensembler.potentials.OneD import harmonicOscillator, envelopedPotential as envelopedPotential
from ensembler.integrator.newtonian import positionVerletIntegrator
from ensembler.integrator.stochastic import metropolisMonteCarloIntegrator
from ensembler.system.basic_system import system as systems
from ensembler.system.eds_system import edsSystem

In [ ]:
#BUILD
##Ensemble Settings:
s_values = np.logspace(start=0, stop=-0.5, num=4)
trials=20
steps_between_trials=10#0

##Potential:
pot_diff= 0
space_shift=4
k1 = k2 = 1
potentialA = harmonicOscillator(k=k1)
potentialB = harmonicOscillator(k=k2, x_shift=space_shift, y_shift=pot_diff)
potential = envelopedPotential(V_is=[potentialA, potentialB])

###plot Potential
pos = np.linspace(-10,10,100)
for s in s_values:
    potential.s=s
    ene = potential.ene(pos)
    plt.plot(pos, ene)
plt.ylim([min(ene), min(ene)+10])
###plot Potential
pos = np.linspace(-10,10,100)
for s in s_values:
    potential.s=s
    ene = potential.ene(pos)
    plt.plot(pos, ene)
plt.ylim([min(ene), min(ene)+10])


##Integrator
integrator = metropolisMonteCarloIntegrator(randomnessIncreaseFactor=1)

##System
eds_system = edsSystem(integrator=integrator, potential=potential, position=5)


##Ensemble
from ensembler.ensemble import replica_exchange
ensemble = replica_exchange.ReplicaExchangeEnvelopingDistributionSampling(system=eds_system, exchange_criterium=None, s_range=s_values, steps_between_trials=steps_between_trials)

[replica.s for repID, replica in ensemble.replicas.items()]

In [ ]:
reps = ensemble.replicas

for repID, replica in reps.items():
    print(repID)
    replica.simulate(1000, withdrawTraj=True)
    

In [ ]:
trajs = ensemble.get_trajectories()

positions = np.linspace(-10,10)
fig, axes = plt.subplots(ncols=len(trajs), figsize=[16,9])

for traj, ax in zip(trajs, axes):
    ax.plot(positions, ensemble.replicas[traj].potential.ene(positions), c="grey")
    ax.scatter(trajs[traj].position, trajs[traj].totEnergy)
    min_e = min(trajs[traj].totEnergy)
    ax.set_ylim([min_e,min_e+10])
    ax.set_title("replica"+str(traj))


In [ ]:
ensemble.simulate(trials, reset_ensemble=True)

In [ ]:
trajs = ensemble.get_trajectories()
equilib = 10
positions = np.linspace(-10,10)
fig, axes = plt.subplots(ncols=len(trajs), figsize=[16,9])

for traj, ax in zip(trajs, axes):
    ax.plot(positions, ensemble.replicas[traj].potential.ene(positions), c="grey")
    ax.scatter(trajs[traj].position[equilib:], trajs[traj].totEnergy[equilib:])
    min_e = min(trajs[traj].totEnergy)
    ax.set_ylim([min_e,min_e+10])
    ax.set_title("replica"+str(traj))


In [ ]:
plt.figure(figsize=[16,9])
plt.plot(trajs[0].position[equilib:])
plt.plot(trajs[1].position[equilib:])
plt.plot(trajs[2].position[equilib:])

plt.vlines([steps_between_trials*x for x in range(20)], ymin=8, ymax=-10, label="exchange", lw=0.1)
plt.hlines(5, xmin=0,xmax=250, lw=0.5, color="C5", zorder=-1)
plt.hlines(0, xmin=0,xmax=250, lw=0.5, color="C5", zorder=-1)

plt.title("Positions")
plt.ylabel("positions")
plt.xlabel("time")

pass

In [ ]:
trials=20

fig, ax = plt.subplots(ncols=1, figsize=[16,9])
enes = [trajs[x].totEnergy for x in sorted(trajs)] 
min_e = np.min(enes)
max_e = np.max(enes)

ax.vlines([steps_between_trials*x for x in range(trials)], ymin=1.1*min_e, ymax=1.1*max_e, label="exchange", lw=0.1)

for ind, ene in enumerate(enes):
    ax.plot(ene, label="replica"+str(ind))
ax.set_title("replica potE timeseries")
ax.set_ylim(1.1*min_e, 1.1*max_e)
ax.legend()

In [ ]:
stats= ensemble.exchange_information
replicas = np.unique(ensemble.exchange_information.replicaID)
trials = np.unique(ensemble.exchange_information.nExchange)

import itertools as it

fig, ax = plt.subplots(ncols=1, figsize=[16,9])

replica_positions = {}
for replica in replicas:
    replica_positions.update({replica: stats.loc[stats.replicaID==replica].replicaPositionI})

    x = trials
    y = replica_positions[replica]

    ax.plot(x,y , label="replica_"+str(replica))

#plt.yticks(replicas+1, reversed(replicas+1))
print(replicas)
ax.set_yticks(ticks=replicas)
ax.set_yticklabels(labels=replicas)

ax.set_ylabel("replica Positions")
ax.set_xlabel("trials")
ax.set_title("Replica exchange transitions")
if(len(replicas)<10): plt.legend()